# Imports

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
from pandas.testing import assert_frame_equal

In [ ]:
if "notebooks" in os.getcwd():
    %cd ..
scenario = "DE"
pyear = 2040

fn_n = Path(f"resources/tyndp/networks/base_s_all___{pyear}_brownfield.nc")
fn_raw = Path(
    f"data/tyndp_2024_bundle/Demand Profiles/{scenario}/{pyear}/ELECTRICITY_MARKET {scenario} {pyear}.xlsx"
)

# Read network

In [ ]:
n = pypsa.Network(fn_n)

In [ ]:
n.loads_t.p_set.head()

# Read raw file

In [ ]:
data = pd.read_excel(
    fn_raw,
    skiprows=11 if scenario != "NT" else 7,
    index_col=0,
    usecols=lambda name: name == "Date" or name == 2009,
    sheet_name=None,
)

In [ ]:
data["PL00"].index = data["AL00"].index
data["UK00"] = pd.read_excel(
    fn_raw,
    skiprows=11,
    index_col=0,
    usecols=lambda name: name == "Date" or name == 2009 - 1,
    sheet_name="UK00",
)

In [ ]:
demand = pd.concat(data, axis=1).droplevel(1, axis=1)

In [ ]:
demand.index = demand.index.map(lambda t: t.replace(year=2013))

demand.head()

In [ ]:
aggregation_map = (
    pd.Series(n.snapshot_weightings.index.get_indexer(demand.index), index=demand.index)
    .replace(-1, np.nan)
    .ffill()
    .astype(int)
    .map(lambda i: n.snapshot_weightings.index[i])
)
demand_agg = demand.groupby(aggregation_map).mean()
demand_agg.columns = demand.columns.str.replace("UK", "GB")
demand_agg.index.name = "snapshot"
demand_agg.columns.name = "Load"

demand_agg.head()

# Compare

In [ ]:
assert_frame_equal(n.loads_t.p_set, demand_agg, check_like=True)